This is a simple example of SimSwap on processing video with multiple faces. You can change the codes for inference based on our other scripts for image or single face swapping.

Code path: https://github.com/neuralchen/SimSwap

Paper path: https://arxiv.org/pdf/2106.06340v1.pdf or https://dl.acm.org/doi/10.1145/3394171.3413630

In [1]:
## make sure you are using a runtime with GPU
## you can check at Runtime/Change runtime type in the top bar.
!nvidia-smi

Tue Aug 30 10:33:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installation

All file changes made by this notebook are temporary. 
You can try to mount your own google drive to store files if you want.


In [2]:
#(repositorio original)
#!git clone https://github.com/neuralchen/SimSwap 
#!cd SimSwap && git pull
#(mi repositorio con  GUI 512, opciones y preparado para 512)
!git clone https://github.com/alu0100585704/simswap-main.git 
!cd simswap-main && git pull

Cloning into 'simswap-main'...
remote: Enumerating objects: 5151, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 5151 (delta 1), reused 5 (delta 1), pack-reused 5140
Receiving objects: 100% (5151/5151), 466.91 MiB | 17.31 MiB/s, done.
Resolving deltas: 100% (1168/1168), done.
Checking out files: 100% (5031/5031), done.
Already up to date.


In [3]:
!pip install insightface==0.2.1 onnxruntime moviepy
!pip install googledrivedownloader
!pip install imageio==2.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 54.7 MB/s 
     |████████████████████████████████| 13.1 MB 42.9 MB/s 
     |████████████████████████████████| 46 kB 4.4 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 20.4 MB/s 
  Created wheel for imageio: filename=imageio-2.4.1-py3-none-any.whl size=3303885 sha256=ecf65725471cb3f530cc05ff190dc978a6ddcc23f23d2ee1979edb0859bd49bd
  Stored in directory: /root/.cache/pip/wheels/46/20/07/7bb9c8c44e6ec2efa60fd0e6280094f53f65f41767ef69a5ee
Successfully built imageio
  Attempting uninstall: imageio
    Found existing installation: imageio 2.9.0
    Uninstalling imageio-2.9.0:
      Successfull

In [4]:
import os
#os.chdir("SimSwap")
os.chdir("simswap-main")
!ls

 actualizar.cmd		   README.md
 cog.yaml		   requirements.txt
 crop_224		   save.gif
 data			   SimSwap512.exe
 demo_file		  'SimSwap colab.ipynb'
 docs			   simswap_colab.py
 download-weights.sh	   simswaplogo
 ejecutar.cmd		   test_one_image.py
 ejecutar_gpu.bat	   test_video_swapmulti.py
'enlaces de interes.txt'   test_video_swap_multispecific.py
 entorno.cmd		   test_video_swapsingle.py
 gpu.py			   test_video_swapspecific_instant.py
 hacer_portable		   test_video_swapspecific.py
 insightface_func	   test_wholeimage_swapmulti.py
 instalar.cmd		   test_wholeimage_swap_multispecific.py
 LICENSE		   test_wholeimage_swapsingle.py
 models			   test_wholeimage_swapspecific.py
 MultiSpecific.ipynb	   train.ipynb
 options		   train.py
 output			   unzip.exe
 parsing_model		   util
 pg_modules		   vb6
 predict.py		  'video explicativo GUI.txt'
 Python38		   wget.exe


In [ ]:
from google_drive_downloader import GoogleDriveDownloader

### it seems that google drive link may not be permenant, you can find this ID from our open url.
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1TLNdIufzwesDbyr_nVTR7Zrx9oRHLM_N',
#                                     dest_path='./arcface_model/arcface_checkpoint.tar')
# GoogleDriveDownloader.download_file_from_google_drive(file_id='1PXkRiBUYbu1xWpQyDEJvGKeqqUFthJcI',
#                                     dest_path='./checkpoints.zip')

!wget -P ./arcface_model https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!unzip ./checkpoints.zip  -d ./checkpoints

!wget https://github.com/neuralchen/SimSwap/releases/download/512_beta/512.zip
!unzip ./512.zip  -d ./checkpoints

!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth

In [6]:
## You can upload filed manually
# from google.colab import drive
# drive.mount('/content/gdrive')

### Now onedrive file can be downloaded in Colab directly!
### If the link blow is not permanent, you can just download it from the 
### open url(can be found at [our repo]/doc/guidance/preparation.md) and copy the assigned download link here.
### many thanks to woctezuma for this very useful help
!wget --no-check-certificate "https://sh23tw.dm.files.1drv.com/y4mmGiIkNVigkSwOKDcV3nwMJulRGhbtHdkheehR5TArc52UjudUYNXAEvKCii2O5LAmzGCGK6IfleocxuDeoKxDZkNzDRSt4ZUlEt8GlSOpCXAFEkBwaZimtWGDRbpIGpb_pz9Nq5jATBQpezBS6G_UtspWTkgrXHHxhviV2nWy8APPx134zOZrUIbkSF6xnsqzs3uZ_SEX_m9Rey0ykpx9w" -O antelope.zip
!unzip ./antelope.zip -d ./insightface_func/models/


--2022-08-30 10:39:58--  https://sh23tw.dm.files.1drv.com/y4mmGiIkNVigkSwOKDcV3nwMJulRGhbtHdkheehR5TArc52UjudUYNXAEvKCii2O5LAmzGCGK6IfleocxuDeoKxDZkNzDRSt4ZUlEt8GlSOpCXAFEkBwaZimtWGDRbpIGpb_pz9Nq5jATBQpezBS6G_UtspWTkgrXHHxhviV2nWy8APPx134zOZrUIbkSF6xnsqzs3uZ_SEX_m9Rey0ykpx9w
Resolving sh23tw.dm.files.1drv.com (sh23tw.dm.files.1drv.com)... 13.107.42.12
Connecting to sh23tw.dm.files.1drv.com (sh23tw.dm.files.1drv.com)|13.107.42.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248024513 (237M) [application/zip]
Saving to: ‘antelope.zip’

antelope.zip        100%[===================>] 236.53M  19.6MB/s    in 21s     

2022-08-30 10:40:20 (11.3 MB/s) - ‘antelope.zip’ saved [248024513/248024513]

Archive:  ./antelope.zip
   creating: ./insightface_func/models/antelope/
  inflating: ./insightface_func/models/antelope/glintr100.onnx  
  inflating: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx  


## Inference

In [17]:
import cv2
import torch
import fractions
import numpy as np
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from models.models import create_model
from options.test_options import TestOptions
from insightface_func.face_detect_crop_multi import Face_detect_crop
from util.add_watermark import watermark_image
from util.videoswap import video_swap


/content/simswap-main


In [ ]:
transformer = transforms.Compose([
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

transformer_Arcface = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

detransformer = transforms.Compose([
        transforms.Normalize([0, 0, 0], [1/0.229, 1/0.224, 1/0.225]),
        transforms.Normalize([-0.485, -0.456, -0.406], [1, 1, 1])
    ])

In [ ]:
opt = TestOptions()
opt.initialize()
opt.parser.add_argument('-f') ## dummy arg to avoid bug
opt = opt.parse()
opt.pic_a_path = './demo_file/Iron_man.jpg' ## or replace it with image from your own google drive
opt.video_path = './demo_file/multi_people_1080p.mp4' ## or replace it with video from your own google drive
opt.output_path = './output/demo.mp4'
opt.temp_path = './tmp'
opt.Arc_path = './arcface_model/arcface_checkpoint.tar'
opt.isTrain = False
opt.use_mask = True  ## new feature up-to-date
opt.no_simswaplogo = True

opt.crop_size = 512

pic_a_path = input("Imagen origen por defecto: " + opt.pic_a_path + "\nIntroduzca ruta con nomnbre de nueva imagen o presiona ENTER para aceptar: ") 
if len(pic_a_path) > 0:    
    opt.pic_a_path = pic_a_path

video_path = input("Video origen por defecto: " + opt.video_path + "\nIntroduzca ruta con nombre de nuevo Video o presiona ENTER para aceptar: ") 
if len(video_path) > 0:    
    opt.video_path = video_path


output_path = input("Video destino por defecto: " + opt.output_path + "\nIntroduzca ruta con nombre de nuevo video de destiono o presiona ENTER para aceptar: ") 
if len(output_path) > 0:    
    opt.output_path = output_path

crop_size = input("Crop Size por defecto: " + str(opt.crop_size) + "\nIntroduzca nuevo valor 224 o presiona ENTER para continuar con"  + str(opt.crop_size) +": ") 
if len(crop_size) > 0:    
    opt.crop_size = int(crop_size)


if len(input("Introduzca cualquier valor para Poner marca de agua o presiona ENTER para continuar sin marca de agua: ")) > 0:    
    opt.no_simswaplogo = False

if len(input("Introduzca cualquier valor para desactivar máscaras o presiona ENTER para continuar con Máscara activada: ")) > 0:    
    opt.use_mask = False

crop_size = opt.crop_size


torch.nn.Module.dump_patches = True

if crop_size == 512:
   opt.which_epoch = 550000
   opt.name = '512'
   mode = 'ffhq'
else:
   mode = 'None'

model = create_model(opt)
model.eval()

app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640),mode=mode)

with torch.no_grad():
    pic_a = opt.pic_a_path
    # img_a = Image.open(pic_a).convert('RGB')
    img_a_whole = cv2.imread(pic_a)
    img_a_align_crop, _ = app.get(img_a_whole,crop_size)
    img_a_align_crop_pil = Image.fromarray(cv2.cvtColor(img_a_align_crop[0],cv2.COLOR_BGR2RGB)) 
    img_a = transformer_Arcface(img_a_align_crop_pil)
    img_id = img_a.view(-1, img_a.shape[0], img_a.shape[1], img_a.shape[2])

    # convert numpy to tensor
    img_id = img_id.cuda()

    #create latent id
    img_id_downsample = F.interpolate(img_id, size=(112,112))
    latend_id = model.netArc(img_id_downsample)
    latend_id = latend_id.detach().to('cpu')
    latend_id = latend_id/np.linalg.norm(latend_id,axis=1,keepdims=True)
    latend_id = latend_id.to('cuda')

    video_swap(opt.video_path, latend_id, model, app, opt.output_path, temp_results_dir=opt.temp_path, use_mask=opt.use_mask,no_simswaplogo=opt.no_simswaplogo,crop_size=crop_size)

------------ Options -------------
Arc_path: models/BEST_checkpoint.tar
aspect_ratio: 1.0
batchSize: 8
checkpoints_dir: ./checkpoints
cluster_path: features_clustered_010.npy
data_type: 32
dataroot: ./datasets/cityscapes/
display_winsize: 512
engine: None
export_onnx: None
f: /root/.local/share/jupyter/runtime/kernel-6d955151-4911-464a-824d-f0806d8071f6.json
feat_num: 3
fineSize: 512
fp16: False
gpu_ids: [0]
how_many: 50
image_size: 224
input_nc: 3
instance_feat: False
isTrain: False
label_feat: False
label_nc: 0
latent_size: 512
loadSize: 1024
load_features: False
local_rank: 0
max_dataset_size: inf
model: pix2pixHD
nThreads: 2
n_blocks_global: 6
n_blocks_local: 3
n_clusters: 10
n_downsample_E: 4
n_downsample_global: 3
n_local_enhancers: 1
name: people
nef: 16
netG: global
ngf: 64
niter_fix_global: 0
no_flip: False
no_instance: False
norm: batch
norm_G: spectralspadesyncbatch3x3
ntest: inf
onnx: None
output_nc: 3
output_path: ./output/
phase: test
pic_a_path: ./crop_224/gdg.jpg
pic_b_

  0%|          | 0/594 [00:00<?, ?it/s]

(142, 366, 4)


100%|██████████| 594/594 [08:45<00:00,  1.13it/s]


[MoviePy] >>>> Building video ./output/demo.mp4
[MoviePy] Writing audio in demoTEMP_MPY_wvf_snd.mp3


100%|██████████| 438/438 [00:00<00:00, 877.18it/s]


[MoviePy] Done.
[MoviePy] Writing video ./output/demo.mp4


100%|██████████| 595/595 [00:53<00:00, 11.15it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output/demo.mp4 

